In [101]:
from itertools import combinations

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.stats import mannwhitneyu
import seaborn as sns

In [25]:
meta = pd.read_csv('../data/processed/toil_tcga_sample_info_140722.csv')

In [5]:
annot = pd.read_csv('../data/processed/ensembl_annotation_050722.csv', low_memory=False)

In [37]:
def tcga_group(ctype, iscancer=True):
    if iscancer:
        return meta[(meta['project'] == ctype) & (meta['sample_type'] == 'primary tumor')]['sample'].to_list()
    else:
        return meta[(meta['project'] == ctype) & (meta['sample_type'] == 'normal tissue')]['sample'].to_list()

In [66]:
def prefilter(df):
    df = df.reset_index()
    
    df_gene_median_tpm = df.drop('ensembl_trs_id', axis=1).groupby('ensembl_gene_id').sum().replace(0.0, np.nan).median(0, skipna=True).mean()
    
    trs_tpm_sum = df.drop('ensembl_gene_id', axis=1).set_index('ensembl_trs_id').max(1)
    
    df = df[df['ensembl_trs_id'].isin(trs_tpm_sum[trs_tpm_sum > 3].index)]
    
    genes_n_trs = df.groupby('ensembl_gene_id').size()
    genes_tpm_sum = df.drop('ensembl_trs_id', axis=1).groupby('ensembl_gene_id').sum().max(1)
    
    genes_ok = (genes_n_trs[genes_n_trs > 1].index.intersection(genes_tpm_sum[genes_tpm_sum > df_gene_median_tpm].index))
    
    df = df[df['ensembl_gene_id'].isin(genes_ok)]
    
    df = df.set_index(['ensembl_gene_id', 'ensembl_trs_id'])     
    
    return df

In [ ]:
def iso_plot():
    pass

In [102]:
def analyze_prevalence(data, ctype):
    res = pd.DataFrame(columns=['ensembl_gene_id', 'trs_1', 'trs_2', 'mean_norm', 'std_norm', 'mean_cancer', 'std_cancer', 'score', 'mwu_pval'])
    
    for i, (gene, part) in enumerate(data.groupby(level=0)):
        if len(part) < 2:
            pass
        
        part = part.droplevel(0) 
        
        for t1, t2 in combinations(part.index, 2):
            nx = np.cos(-np.pi/4) * part.loc[t1, tcga_group(ctype, False)].to_numpy() + np.sin(-np.pi/4) * part.loc[t2, tcga_group(ctype, False)].to_numpy()
            
            cx = np.cos(-np.pi/4) * part.loc[t1, tcga_group(ctype)].to_numpy() + np.sin(-np.pi/4) * part.loc[t2, tcga_group(ctype)].to_numpy()
            
            nx_mean = np.mean(nx)
            nx_std = np.std(nx)
            
            cx_mean = np.mean(cx)
            cx_std = np.std(cx)
            
            score = nx_mean * cx_mean
            
            mwu = mannwhitneyu(nx, cx)[1]
            
            res.loc[len(res.index)] = [gene, t1, t2, nx_mean, nx_std, cx_mean, cx_std, score, mwu]
            
        print(f'{i} / {len(data.index.get_level_values(0).unique())}    ', end='\r')
            
    return res

In [67]:
data = pd.read_csv('../data/processed/toil/toil_tcga_coad.csv')

In [68]:
data = data.rename({'sample': 'ensembl_trs_id'}, axis=1)

In [69]:
data['ensembl_gene_id'] = data['ensembl_trs_id'].map(annot.set_index('ensembl_trs_id')['ensembl_gene_id'])

In [70]:
data = data.dropna(axis=0, subset='ensembl_gene_id')

In [71]:
data = data.set_index(['ensembl_gene_id', 'ensembl_trs_id']).sort_index()

In [72]:
data = prefilter(data)

In [103]:
r = analyze_prevalence(data.loc[['ENSG00000000003', 'ENSG00000104635']], 'COAD')

In [104]:
r

,ensembl_gene_id,trs_1,trs_2,mean_norm,std_norm,mean_cancer,std_cancer,score,mwu_pval
0,ENSG00000000003,ENST00000373020,ENST00000496771,37.609334,11.291865,52.575750,32.149144,1977.338953,1.024639e-02
1,ENSG00000000003,ENST00000373020,ENST00000612152,36.710112,10.971987,51.273506,31.343782,1882.256149,1.051037e-02
2,ENSG00000000003,ENST00000373020,ENST00000614008,38.091376,11.375713,53.251013,32.437654,2028.404370,9.392419e-03
3,ENSG00000000003,ENST00000496771,ENST00000612152,-0.899222,0.478344,-1.302244,1.256977,1.171007,1.581501e-01
4,ENSG00000000003,ENST00000496771,ENST00000614008,0.482043,0.430218,0.675262,0.667571,0.325505,1.560736e-01
5,ENSG00000000003,ENST00000612152,ENST00000614008,1.381264,0.664074,1.977507,1.468018,2.731460,4.055022e-02
6,ENSG00000104635,ENST00000289952,ENST00000359741,-18.388126,7.327453,-3.621837,4.394899,66.598789,2.257462e-21
7,ENSG00000104635,ENST00000289952,ENST00000381237,-7.846472,3.651358,-24.695977,12.606843,193.776287,2.217774e-20
8,ENSG00000104635,ENST00000289952,ENST00000517370,1.597549,0.891432,0.021066,0.326965,0.033653,2.631370e-23
9,ENSG00000104635,ENST00000289952,ENST00000520644,1.940754,0.998050,0.160427,0.482326,0.311349,5.536366e-23


In [97]:
data.loc[['ENSG00000000003', 'ENSG00000104635']]

TCGA-D5-5538-01  TCGA-F4-6854-01  \
ensembl_gene_id ensembl_trs_id                                      
ENSG00000000003 ENST00000373020        172.07475         98.57059   
                ENST00000496771          6.40978          3.58009   
                ENST00000612152          5.05011          5.31006   
                ENST00000614008          1.66006          0.53001   
ENSG00000104635 ENST00000289952          0.36001          0.41000   
                ENST00000359741          3.18998          6.30008   
                ENST00000381237         28.28076         33.55968   
                ENST00000517370          0.20000          0.38999   
                ENST00000520644          0.00000          0.20000   

                                 TCGA-AA-3511-01  TCGA-QG-A5YX-01  \
ensembl_gene_id ensembl_trs_id                                      
ENSG00000000003 ENST00000373020         76.28783        114.15002   
                ENST00000496771          0.72998          0.92002   
                ENST00000612152          2.85996          2.99991   
                ENST00000614008          0.00000          0.27001   
ENSG00000104635 ENST00000289952          0.80001          0.64999   
                ENST00000359741          7.06993         16.55979   
                ENST00000381237         48.78854         42.59069   
                ENST00000517370          0.75001          0.53001   
                ENST00000520644          0.00000          0.00000   

                                 TCGA-G4-6320-01  TCGA-CM-6164-01  \
ensembl_gene_id ensembl_trs_id                                      
ENSG00000000003 ENST00000373020         55.81860        139.30369   
                ENST00000496771          1.11003          1.88000   
                ENST00000612152          2.93999          5.81005   
                ENST00000614008          0.00000          0.00000   
ENSG00000104635 ENST00000289952          0.04000          0.54000   
                ENST00000359741          1.97996          4.42008   
                ENST00000381237         19.43016         54.81025   
                ENST00000517370          0.10000          0.58999   
                ENST00000520644          0.00000          0.00000   

                                 TCGA-A6-A567-01  TCGA-AA-3516-11  \
ensembl_gene_id ensembl_trs_id                                      
ENSG00000000003 ENST00000373020         63.38973         44.71130   
                ENST00000496771          1.64003          0.33999   
                ENST00000612152          2.33996          1.90005   
                ENST00000614008          0.00000          0.00000   
ENSG00000104635 ENST00000289952          0.50001          3.21998   
                ENST00000359741          0.81001         36.48067   
                ENST00000381237         29.19907          7.20001   
                ENST00000517370          0.22000          0.27999   
                ENST00000520644          0.00000          0.00000   

                                 TCGA-DM-A0X9-01  TCGA-AY-A69D-01  ...  \
ensembl_gene_id ensembl_trs_id                                     ...   
ENSG00000000003 ENST00000373020         88.92294        130.66138  ...   
                ENST00000496771          0.70999          4.83998  ...   
                ENST00000612152          3.42993          6.84993  ...   
                ENST00000614008          0.10000          0.57001  ...   
ENSG00000104635 ENST00000289952          0.07000          1.46004  ...   
                ENST00000359741          6.06019          7.73024  ...   
                ENST00000381237         42.77116         46.83026  ...   
                ENST00000517370          0.47000          1.96001  ...   
                ENST00000520644          0.00000          0.00000  ...   

                                 TCGA-AA-3514-11  TCGA-DM-A282-01  \
ensembl_gene_id ensembl_trs_id                                      
ENSG00000000003 ENST00000373020         